# Herramienta de visualización
**Facundo A. Lucianna - Inteligencia Artificial - CEIA - FIUBA**

Un extra que se presenta aquí es básicamente aplicar la solución encontrada por nuestro algoritmo, para ello se creó un visualizador basado en [PyGame](https://www.pygame.org/news) que obtiene un representación de las Torres de Hanoi y realiza la secuencia que nos indique la solución encontrada.

![files](./img/hanoi_sim.png)

La implementación de esto está en `simulator` y está fuertemente documentado por si les da curiosidad cómo se implementó. Es altamente configurable, tanto en velocidad general como en tamaños y geometrías, todos accesibles desde `constants.py`.

Para ejecutar la animación script, desde la línea de comando se debe utilizar el siguiente comando:

```bash
python3 ./simulation_hanoi.py

```

Para su funcionamiento, el visualizador depende de dos archivos JSON que deben encontrarse dentro de `simulator`:

- `initial_state.json`: Este JSON indica cómo se inicializan los discos y determina cuántos discos habrá en total.
- `sequence.json`: Este JSON indica el orden de los movimientos de los discos. 

El visualizador permite movimientos ilegales dentro del juego de movimientos de discos, y siempre recuerda la altura a la que debe quedar el disco al insertarse en una varilla específica. Sin embargo, no puede realizar movimientos imposibles, como mover un disco que no esté en la varilla correcta o sacar el último disco de una varilla llena. En sí, el visualizador no fallará, pero la animación resultante puede no tener sentido en estos casos.

## Requisitos para los programas de búsqueda de soluciones

Los programas utilizados para encontrar soluciones deben respetar las siguientes especificaciones para que la solución pueda ser visualizada correctamente en el visualizador.

### `initial_state.json`

El programa de búsqueda debe generar (o pueden crear manualmente) un JSON con el estado inicial de la ubicación de los discos. Se aceptan cualquier configuración y número de discos (máximo 15 discos).

El archivo tiene el siguiente formato:

```JSON
{
  "peg_1": [5, 4, 3, 2, 1],
  "peg_2": [],
  "peg_3": []
}
```

Donde `peg_1` es la varilla de la izquierda, `peg_2` es la varilla del medio y `peg_3` es la varilla de la derecha. Para cada varilla se guarda un array con los números correspondientes a cada disco, siendo el 1 el más pequeño y así sucesivamente.

Se acepta cualquier configuración siempre y cuando **no se repita** ningún número de disco y todos los discos estén presentes (por ejemplo, si está el disco 4, también deben estar presentes el 1, el 2 y el 3).

Este es otro ejemplo válido:

```JSON
{
  "peg_1": [6, 2],
  "peg_2": [8, 7, 4],
  "peg_3": [1, 5, 3]
}
```

❌ Este es invalido porque falta el disco 4

```JSON
{
  "peg_1": [6, 2],
  "peg_2": [8, 7],
  "peg_3": [1, 5, 3]
}
```

❌ Este es inválido porque el disco 6 está repetido dos veces

```JSON
{
  "peg_1": [6, 2],
  "peg_2": [8, 7, 6],
  "peg_3": [1, 5, 3]
}
```

### `sequence.json`

El programa de búsqueda debe generar un JSON con la secuencia de movimientos de un disco por vez. 

El archivo tiene el siguiente formato:

```JSON
[
	{
		"type": "movement",
		"disk": 1,
		"peg_start": 1,
		"peg_end": 2
	},
	{
		"type": "movement",
		"disk": 2,
		"peg_start": 1,
		"peg_end": 3
	},
        .
        .
        . 
	{
		"type": "movement",
		"disk": 3,
		"peg_start": 2,
		"peg_end": 1
	}
]
```

Se observa que es un array de objetos JSON.

Cada movimiento indica qué disco (`disk`) se mueve, desde qué varilla (`peg_start`) y hacia qué varilla se inserta (`peg_end`). El script ejecuta el movimiento si es de tipo `movement`. Otros tipos de secuencias son ignorados por el script.

---

## Usando la implementación que vimos en notebooks anteriores

La implementación de la clase `NodeHanoi` para armar los nodos del árbol de búsqueda tiene un método llamado `generate_solution_for_simulator()` el cual automáticamente genera los dos archivos con toda la secuencia desde la raíz hasta el nodo que se está ejecutando el método.

Traigamos el algoritmo de búsqueda que implementamos en el anterior notebook:

In [105]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi
from aima_libs.aima import PriorityQueue as AimaPriorityQueue

index = 0

In [210]:
import random

def priority_func(x: NodeHanoi):
    return x

In [211]:
def breadth_first_search(number_disks=5):
    # Inicializamos el problema
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    # Creamos una cola FIFO con el nodo inicial
    frontier =  AimaPriorityQueue(order='min', f=priority_func)
    
    #node = NodeHanoi(problem.initial)  # Nodo inicial   
    frontier.append(NodeHanoi(problem.initial))
    
    # Creamos el set con estados ya visitados
    explored = set()
    
    node_explored = 0
    
    while len(frontier) != 0:
        node = frontier.pop()
        node_explored += 1
        
        # Agregamos el estado del nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
        explored.add(node[1].state)
        
        if problem.goal_test(node[1].state):  # Comprobamos si hemos alcanzado el estado objetivo
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node[1].depth,
                "cost_total": node[1].state.accumulated_cost,
            }
            return node[1], metrics
        
        # Agregamos a la cola todos los nodos sucesores del nodo actual
        for next_node in node[1].expand(problem):
            # Solo si el estado del nodo no fue explorado
            if next_node.state not in explored:               
                frontier.append(next_node)  # Corregido

    # Si no se encontro la solución, devolvemos la métricas igual
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth, # OBS: Si no se encontró la solución, este valor solo tiene sentido en breadth_first_search, en otros casos se debe ir llevando registro de cual fue la máxima profundidad
        "cost_total": None,
    }
    return None, metrics


Ejecutemos una búsqueda:

In [212]:
solution, metrics = breadth_first_search(number_disks=5)

Veamos algunas metricas:

In [213]:
for key, value in metrics.items():
    print(f"{key}: {value}")

Ahora ejecutemos el método que nos genera los archivos para el visualizador:

In [214]:
solution.generate_solution_for_simulator()

El método no retorna nada, pero en donde ejecutamos esta notebook se guardaron los dos archivos.

![files](./img/files.png)

Esos `JSONs` los movemos a la carpeta `simulator` y desde esa carpeta podemos ejecutar:


```bash

conda init powershell #en el powershell para poder utilizar los comandos de conda y poder cambiar de enviroment

(base) conda info --envs  #para ver los entornos disponibles

# conda environments:
#
                       C:\Users\mpaur\anaconda3
                       C:\Users\mpaur\anaconda3\envs\EntornoIA
base                 * C:\Users\mpaur\miniconda3


(base)conda activate C:\Users\mpaur\anaconda3\envs\EntornoIA

(EntornoIA) python ./simulation_hanoi.py

```

Para poder visualizar la solución encontrada.